# Test progress bars

## ipywidgets

In [1]:
import ipywidgets as widgets

In [2]:
widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

IntProgress(value=7, description='Loading:', max=10)

In [3]:
widgets.FloatProgress(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)

FloatProgress(value=7.5, bar_style='info', description='Loading:', max=10.0)

## astropy

In [4]:
import time
from astropy.utils.console import ProgressBar

In [5]:
for i in ProgressBar(range(10)):
    time.sleep(0.1)

In [6]:
for i in ProgressBar(range(10), ipython_widget=True):
    time.sleep(0.1)

FloatProgress(value=0.0)

## tqdm

In [7]:
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [8]:
for i in tqdm(range(10)):
    time.sleep(0.1)

100%|██████████| 10/10 [00:01<00:00,  9.77it/s]


In [9]:
for i in tqdm_notebook(range(10)):
    time.sleep(0.1)

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

## With Joblib

From https://gist.github.com/MInner/12f9cf961059aed1a60e72c5531a697f

In [1]:
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed
import time

import random

def func(x):
    time.sleep(random.randint(1, 10))
    return x

def text_progessbar(seq, total=None):
    step = 1
    tick = time.time()
    while True:
        time_diff = time.time()-tick
        avg_speed = time_diff/step
        total_str = 'of %n' % total if total else ''
        print('step', step, '%.2f' % time_diff, 'avg: %.2f iter/sec' % avg_speed, total_str)
        step += 1
        yield next(seq)

all_bar_funcs = {
    'tqdm': lambda args: lambda x: tqdm(x, **args),
    'txt': lambda args: lambda x: text_progessbar(x, **args),
    'False': lambda args: iter,
    'None': lambda args: iter,
}

def ParallelExecutor(use_bar='tqdm', **joblib_args):
    def aprun(bar=use_bar, **tq_args):
        def tmp(op_iter):
            if str(bar) in all_bar_funcs.keys():
                bar_func = all_bar_funcs[str(bar)](tq_args)
            else:
                raise ValueError("Value %s not supported as bar type"%bar)
            return Parallel(**joblib_args)(bar_func(op_iter))
        return tmp
    return aprun

aprun = ParallelExecutor(n_jobs=5)

a1 = aprun(total=25)(delayed(func)(i ** 2 + j) for i in range(5) for j in range(5))
a2 = aprun(total=16)(delayed(func)(i ** 2 + j) for i in range(4) for j in range(4))
a2 = aprun(bar='txt')(delayed(func)(i ** 2 + j) for i in range(4) for j in range(4))
a2 = aprun(bar=None)(delayed(func)(i ** 2 + j) for i in range(4) for j in range(4))

HBox(children=(IntProgress(value=0, max=25), HTML(value='')))

HBox(children=(IntProgress(value=0, max=16), HTML(value='')))


step 1 0.00 avg: 0.00 iter/sec 
step 2 0.00 avg: 0.00 iter/sec 
step 3 0.00 avg: 0.00 iter/sec 
step 4 0.00 avg: 0.00 iter/sec 
step 5 0.00 avg: 0.00 iter/sec 
step 6 0.00 avg: 0.00 iter/sec 
step 7 0.00 avg: 0.00 iter/sec 
step 8 0.00 avg: 0.00 iter/sec 
step 9 0.00 avg: 0.00 iter/sec 
step 10 0.00 avg: 0.00 iter/sec 
step 11 6.00 avg: 0.55 iter/sec 
step 12 6.01 avg: 0.50 iter/sec 
step 13 7.01 avg: 0.54 iter/sec 
step 14 7.01 avg: 0.50 iter/sec 
step 15 9.01 avg: 0.60 iter/sec 
step 16 11.01 avg: 0.69 iter/sec 
step 17 12.01 avg: 0.71 iter/sec 


In [4]:
Parallel(n_jobs=2, verbose=11)(delayed(func)(i ** 2) for i in range(5))

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    3.0s
[Parallel(n_jobs=2)]: Done   2 tasks      | elapsed:    8.0s
[Parallel(n_jobs=2)]: Done   3 out of   5 | elapsed:    9.0s remaining:    6.0s
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   12.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   12.0s finished


[0, 1, 4, 9, 16]

In [6]:
with tqdm(total=10) as pbar:
    for i in range(10):
        time.sleep(0.1)
        pbar.update(1)

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [10]:
for i in tqdm(range(10)):
    time.sleep(0.1)

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [39]:
def func(x):
    time.sleep(.1)
    return x
    
res = Parallel(n_jobs=4, verbose=0)(tqdm((delayed(func)(i ** 2) for i in range(100)), total=100))

res = Parallel(n_jobs=1, verbose=0)(tqdm((delayed(func)(i ** 2) for i in range(10)), total=10))

HBox(children=(IntProgress(value=0), HTML(value='')))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))